<a href="https://colab.research.google.com/github/hansglick/book_errata/blob/main/hf_transformers_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 101 kB 1.9 MB/s 
     |████████████████████████████████| 6.6 MB 37.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Organisation de la librairie

 La librairie `transformers` contient apparemment 5 modules :
  * `transformers`
  * `accelerate`
  * `hub`
  * `datasets`
  * `tokenizers`

 
## Parcours

 * chapitres 1->4 : utliser un modèle pretrained, le finetunner, le partager
 * chapitres 5->8 : introduction aux modules `datasets`, `tokenizers`, et appréhension des problèmes classiques NLP
 * chapitres 9->12 : au delà de la NLP, introduction à la speech recognition et à la computer vision, également introduction aux démonstrateurs huggingface ainsi que de la mise en production


## Pipeline

Il y a cette fonction `pipeline()` dans le module `transformers` qui permet de créer une chaîne qui effectue toutes les étapes suivantes pour un cas d'usage renseigné : 
 * Téléchargement d'un pretrained model adapté au cas d'usage renseigné
 * le texte est prétraité pour qu’il ait un format compréhensible par le modèle,
 * les données prétraitées sont passées au modèle,
 * les prédictions du modèle sont post-traitées de sorte que vous puissiez les comprendre.

```python
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

```

```python
classifier(
    "I've been waiting for a HuggingFace course my whole life."
)  # J'ai attendu un cours d'HuggingFace toute ma vie.

# On peut passer plusieurs observations dans le modèle directement
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]  # « J'ai attendu un cours d'HuggingFace toute ma vie. »,  « Je déteste tellement ça ! »
)
```

Voici une liste non-exhaustive des pipelines disponibles, la liste exhaustive est disponible ici : https://huggingface.co/docs/transformers/main_classes/pipelines

 * `feature-extraction` (pour obtenir la représentation vectorielle d’un texte)
 * `fill-mask`
 * `ner` (named entity recognition ou reconnaissance d’entités nommées en français)
 * `question-answering`
 * `sentiment-analysis`
 * `summarization`
 * `text-generation`
 * `translation`
 * `zero-shot-classification`

#### Zero Shot Learning

Il s'agit de prédire la classe de morceaux de textes alors qu'aucun finetunning n'a été effectué

```python
classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    # C'est un cours sur la bibliothèque Transformers
    candidate_labels=["education", "politics", "business"],
)
```

#### Génération de texte

Il s'agit de prédire la classe de morceaux de textes alors qu'aucun finetunning n'a été effectué. On peut renseigner les arguments `num_return_sequences` et `max_length` afin de spécifier le nombre de séquences retournées ainsi que leur longueur


```python
generator = pipeline("text-generation")
generator(
    "In this course, we will teach you how to"
)  # Dans ce cours, nous vous enseignerons comment
```

``` python
[{'generated_text': 'In this course, we will teach you how to understand and use ' 
					# Dans ce cours, nous vous enseignerons comment comprendre et utiliser
                    'data flow and data interchange when handling user data. We '
					# flux de données et l'échange de données lors de la manipulation des données utilisateur. Nous 
                    'will be working with one or more of the most commonly used ' 
					# travailleront avec un ou plusieurs des plus couramment utilisés
                    'data flows — data flows of various types, as seen by the ' 
					# flux de données - flux de données de différents types, tels qu'ils sont vus par
                    'HTTP'}] # HTTP
```

## Un modèle du Hub dans un pipeline

On peut choisir son modèle dans le Hub de HuggingFace en renseignant l'argument `model`. URL de HFHub : https://huggingface.co/models

```python
generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    # Dans ce cours, nous vous enseignerons comment
    max_length=30,
    num_return_sequences=2,
)
```

```python
[{'generated_text': 'In this course, we will teach you how to manipulate the world and ' 
					# Dans ce cours, nous vous enseignerons comment manipuler le monde et
                    'move your mental and physical capabilities to your advantage.'}, 
					# utiliser vos capacités mentales et physiques à votre avantage.
 {'generated_text': 'In this course, we will teach you how to become an expert and ' 
					# Dans ce cours, nous vous apprendrons comment devenir un expert et
                    'practice realtime, and with a hands on experience on both real ' 
					# pratique en temps réel, et avec une expérience pratique à la fois sur de vrais
                    'time and real'}] 
					# temps et réel
```

### Remplacement fill_mask

La tâche `fill_mask` consiste à prédire les mots manquants dans une phrase.  L'argument `top_k` permet de retourner le top k des mots les plus probables. Le token correspondant au mot masqué n'est pas toujours le même, on doit vérifier son orthographe sur la page du modèle. Par exemple pour le modèle `bert-base-cased` le mot masqué s'écrit `[MASK]`

```python
unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)
```

```python
[{'sequence': 'This course will teach you all about mathematical models.', 
# Ce cours vous apprendra tout sur les modèles mathématiques.
  'score': 0.19619831442832947,
  'token': 30412,
  'token_str': ' mathematical'},
 {'sequence': 'This course will teach you all about computational models.', 
 # Ce cours vous apprendra tout sur les modèles mathématiques.
  'score': 0.04052725434303284,
  'token': 38163,
  'token_str': ' computational'}]
```



### NER
Tâche qui consiste à prédire le tag de chaque mot. L'argument `grouped_entities = True` permet de rassembler en une seule entité deux mots qui se suivent et qui appartiennent à un même tag

```python
ner = pipeline("ner", grouped_entities=True)
ner(
    "My name is Sylvain and I work at Hugging Face in Brooklyn."
)  # Je m'appelle Sylvain et je travaille à Hugging Face à Brooklyn.
```

```python
[{'entity_group': 'PER', 'score': 0.99816, 'word': 'Sylvain', 'start': 11, 'end': 18}, 
 {'entity_group': 'ORG', 'score': 0.97960, 'word': 'Hugging Face', 'start': 33, 'end': 45}, 
 {'entity_group': 'LOC', 'score': 0.99321, 'word': 'Brooklyn', 'start': 49, 'end': 57}
]
```

### Question Answering

Cette tâche consiste à répondre à une question en extrayant la réponse d'un contexte spécifié

```python
question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",  # Où est-ce que je travaille ?
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
    # Je m'appelle Sylvain et je travaille à Hugging Face à Brooklyn.
)
```

```python
{'score': 0.6385916471481323, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}
```

### Summary
La tâche consiste évidemment à résumer un texte long en un texte plus court tout en gardant le sens global. On peut spécifier les arguments suivants : `min_length` et `max_length` afin de d'avoir un résumé de sortie plus ou moins long

```python
summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of 
    graduates in traditional engineering disciplines such as mechanical, civil, 
    electrical, chemical, and aeronautical engineering declined, but in most of 
    the premier American universities engineering curricula now concentrate on 
    and encourage largely the study of engineering science. As a result, there 
    are declining offerings in engineering subjects dealing with infrastructure, 
    the environment, and related issues, and greater concentration on high 
    technology subjects, largely supporting increasingly complex scientific 
    developments. While the latter is important, it should not be at the expense 
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other 
    industrial countries in Europe and Asia, continue to encourage and advance 
    the teaching of engineering. Both China and India, respectively, graduate 
    six and eight times as many traditional engineers as does the United States. 
    Other industrial countries at minimum maintain their output, while America 
    suffers an increasingly serious decline in the number of engineering graduates 
    and a lack of well-educated engineers.
"""
)

"""
    L'Amérique a changé de façon spectaculaire au cours des dernières années. Non seulement le nombre de 
    diplômés dans les disciplines traditionnelles de l'ingénierie telles que le génie mécanique, civil, 
    l'électricité, la chimie et l'aéronautique a diminué, mais dans la plupart 
    des grandes universités américaines, les programmes d'études d'ingénierie se concentrent désormais sur 
    et encouragent largement l'étude des sciences de l'ingénieur. Par conséquent, il y a 
    de moins en moins d'offres dans les sujets d'ingénierie traitant de l'infrastructure, 
    l'environnement et les questions connexes, et une plus grande concentration sur les sujets de haute 
    technologie, qui soutiennent en grande partie des développements scientifiques de plus en plus 
    complexes. Si cette dernière est importante, elle ne doit pas se faire au détriment
    de l'ingénierie plus traditionnelle.

    Les économies en développement rapide telles que la Chine et l'Inde, ainsi que d'autres 
    pays industrialisés d'Europe et d'Asie, continuent d'encourager et de promouvoir
    l'enseignement de l'ingénierie. La Chine et l'Inde, respectivement, diplôment 
    six et huit fois plus d'ingénieurs traditionnels que les États-Unis. 
    Les autres pays industriels maintiennent au minimum leur production, tandis que l'Amérique 
    souffre d'une baisse de plus en plus importante du nombre de diplômés en ingénierie
    et un manque d'ingénieurs bien formés.
"""
```

```python
[{'summary_text': ' America has changed dramatically during recent years . The ' 
				  # L'Amérique a changé de façon spectaculaire au cours des dernières années. Le
                  'number of engineering graduates in the U.S. has declined in ' 
				  # nombre de diplômés en ingénierie aux États-Unis a diminué dans
                  'traditional engineering disciplines such as mechanical, civil ' 
				  # dans les disciplines traditionnelles de l'ingénierie, telles que le génie mécanique, civil
                  ', electrical, chemical, and aeronautical engineering . Rapidly ' 
				  # l'électricité, la chimie et l'aéronautique. Les économies
                  'developing economies such as China and India, as well as other ' 
				  # en développement rapide comme la Chine et l'Inde, ainsi que d'autres
                  'industrial countries in Europe and Asia, continue to encourage ' 
				  # pays industriels d'Europe et d'Asie, continuent d'encourager
                  'and advance engineering.'}] 
				  # et à faire progresser l'ingénierie.
```

### Traduction

Par défaut, on peut choisir un modèle de traduction de la langue A vers la langue b en spécifiant la task `translation_a_to_b`. Le plus simple reste tout de même de choisir un modèle dans le Hub. Pareil pour le résumé, il y a des arguments `min_length` et `max_length` même si je vois pas vraiment dans quel cas ca devrait être intéressant

```python
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")
```

```python
[{'translation_text': 'This course is produced by Hugging Face.'}]
```

## En bref

 * Le `hub` contient les modèles pretrained. On peut également *pousser*
 son propre modèle sur le `hub` et ainsi le rendre disponible à tout le monde